In [1]:
from collections import Counter
import json, random

with open("data/sharded_instructions_600.json", "r") as f:
    data = json.load(f)

Counter([d["task"] for d in data])

num_training = {"data2text": 0, "database": 15, "actions": 15, "math": 15, "code": 15, "summary": 0}

# "sharded-HumanEval/86", "sharded-BFCL/parallel_198", "sharded-BFCL/parallel_167"
train_sample_ids = ["sharded-HumanEval/83", "sharded-HumanEval/76", "sharded-HumanEval/26", "sharded-HumanEval/137", "sharded-HumanEval/36", "sharded-spider-val-798-medium", "sharded-spider-val-129-extra", "sharded-spider-val-748-extra", "sharded-spider-val-492-medium", "sharded-spider-val-401-hard", "sharded-BFCL/parallel_42", "sharded-BFCL/parallel_151", "sharded-BFCL/parallel_186", "sharded-BFCL/parallel_149", "sharded-BFCL/parallel_144", "sharded-GSM8K/435", "sharded-GSM8K/307", "sharded-GSM8K/784", "sharded-GSM8K/1047", "sharded-GSM8K/214"] # "sharded-HumanEval/114", "sharded-HumanEval/86", "sharded-HumanEval/76", "sharded-HumanEval/137", "sharded-HumanEval/36", "sharded-GSM8K/932"
val_sample_ids = ['sharded-spider-val-53-hard', 'sharded-spider-val-671-medium', 'sharded-spider-val-982-extra', 'sharded-spider-val-123-medium', 'sharded-spider-val-281-hard', 'sharded-GSM8K/248', 'sharded-GSM8K/477', 'sharded-GSM8K/929', 'sharded-GSM8K/875', 'sharded-GSM8K/808', 'sharded-livecodebench/2802', 'sharded-HumanEval/7', 'sharded-HumanEval/39', 'sharded-HumanEval/158', 'sharded-HumanEval/74', 'sharded-BFCL/parallel_79', 'sharded-BFCL/parallel_4', 'sharded-BFCL/parallel_92', 'sharded-BFCL/parallel_145', 'sharded-BFCL/parallel_195']
for d in data:
    # d["split"] = "train" if d["task_id"] in train_sample_ids else "test"
    if d["task_id"] in train_sample_ids:
        d["split"] = "train"
    elif d["task_id"] in val_sample_ids:
        d["split"] = "validation"
    else:
        d["split"] = "test"

# for d in data:
#     if d["split"] == "train":
#         print(d["task_id"], len(d["shards"]))

print(Counter([d["split"] for d in data]))
print(f"Number of shards in samples: {Counter([len(d['shards']) for d in data if d['split'] == 'train'])}")

with open("data/sharded_instructions_600.json", "w") as f:
    json.dump(data, f)


Counter({'test': 587, 'train': 20, 'validation': 20})
Number of shards in samples: Counter({4: 11, 5: 9})


In [4]:
import json, random

with open("data/sharded_instructions_600.json", "r") as f:
    data = json.load(f)

tasks = ["database", "math", "code", "actions"]

val_task_ids = []
for task in tasks:
    data_task = [d for d in data if d["task"] == task and d["split"] == "test" and 4 <= len(d["shards"]) <= 5]
    print(task, len(data_task))
    random.shuffle(data_task)

    val_task_ids += [d["task_id"] for d in data_task[:5]]
    
print(val_task_ids)

database 90
math 47
code 16
actions 39
['sharded-spider-val-53-hard', 'sharded-spider-val-671-medium', 'sharded-spider-val-982-extra', 'sharded-spider-val-123-medium', 'sharded-spider-val-281-hard', 'sharded-GSM8K/248', 'sharded-GSM8K/477', 'sharded-GSM8K/929', 'sharded-GSM8K/875', 'sharded-GSM8K/808', 'sharded-livecodebench/2802', 'sharded-HumanEval/7', 'sharded-HumanEval/39', 'sharded-HumanEval/158', 'sharded-HumanEval/74', 'sharded-BFCL/parallel_79', 'sharded-BFCL/parallel_4', 'sharded-BFCL/parallel_92', 'sharded-BFCL/parallel_145', 'sharded-BFCL/parallel_195']


In [18]:
from collections import Counter
import json

tree_logs = []
with open("experiments/exp_20250604_8/tree_logs.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        tree_logs.append(data)

for tree_log in tree_logs:
    print(tree_log["task"].ljust(20), tree_log["task_id"].ljust(40), tree_log["avg_leaf_node_scores"])

code                 sharded-HumanEval/114                    0.0
code                 sharded-HumanEval/139                    0.0
code                 sharded-HumanEval/25                     0.0
code                 sharded-HumanEval/26                     0.0
code                 sharded-HumanEval/43                     0.0
code                 sharded-HumanEval/62                     0.0
code                 sharded-HumanEval/7                      0.0
code                 sharded-HumanEval/71                     0.0
code                 sharded-HumanEval/73                     0.0
code                 sharded-HumanEval/8                      0.0
code                 sharded-HumanEval/83                     0.0
code                 sharded-HumanEval/90                     0.0
code                 sharded-HumanEval/97                     0.0
code                 sharded-HumanEval/98                     0.0
code                 sharded-HumanEval/99                     0.0
database  

In [15]:
import json, numpy as np

with open("data/sharded_instructions_600.json", "r") as f:
    data = json.load(f)

id2sample = {d["task_id"]: d for d in data}

run_logs = []
with open("logs/code/sharded/sharded_code_t-phi-4.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        run_logs.append(data)

task_id_scores = {}
for run_log in run_logs:
    task_id = run_log["task_id"]
    if task_id not in task_id_scores:
        task_id_scores[task_id] = []
    task_id_scores[task_id].append(run_log["score"])

task_id_avg_scores = {task_id: np.mean(scores) for task_id, scores in task_id_scores.items()}

for task_id, score in sorted(task_id_avg_scores.items(), key=lambda x: x[1], reverse=True):
    sample = id2sample[task_id]
    print(task_id, score, len(sample["shards"]))

sharded-HumanEval/44 1.0 5
sharded-HumanEval/25 1.0 6
sharded-HumanEval/8 1.0 6
sharded-HumanEval/7 1.0 5
sharded-HumanEval/43 1.0 6
sharded-HumanEval/97 1.0 5
sharded-livecodebench/2917 1.0 5
sharded-livecodebench/2800 1.0 6
sharded-HumanEval/59 1.0 4
sharded-HumanEval/90 1.0 4
sharded-livecodebench/2847 1.0 8
sharded-livecodebench/2887 0.9 6
sharded-HumanEval/111 0.9 6
sharded-HumanEval/99 0.9 6
sharded-livecodebench/2819 0.9 6
sharded-HumanEval/158 0.9 5
sharded-HumanEval/74 0.9 4
sharded-livecodebench/2844 0.9 7
sharded-livecodebench/2998 0.9 6
sharded-HumanEval/88 0.9 7
sharded-HumanEval/150 0.8 5
sharded-HumanEval/120 0.8 7
sharded-livecodebench/2876 0.8 8
sharded-HumanEval/62 0.8 5
sharded-HumanEval/98 0.7 4
sharded-livecodebench/2845 0.7 6
sharded-livecodebench/2892 0.7 7
sharded-livecodebench/2856 0.7 7
sharded-livecodebench/2727 0.7 10
sharded-HumanEval/63 0.7 7
sharded-HumanEval/73 0.7 6
sharded-HumanEval/124 0.6 9
sharded-livecodebench/2977 0.6 9
sharded-HumanEval/123 0.6 1